In [1]:
%pip install tornado_helper --upgrade --no-cache-dir --find-links=. --force-reinstall --no-deps
%pip install pyproj netcdf4 folium xarray

from importlib import reload
import logging
import sys 

reload(logging)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

Looking in links: .
  Attempting uninstall: tornado_helper
    Found existing installation: tornado_helper 2.1.4
    Uninstalling tornado_helper-2.1.4:
      Successfully uninstalled tornado_helper-2.1.4
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tornado_helper import Combined

combined = Combined() 
df = combined.catalog()
df.head()

INFO:root:GOES initialized at data_goes
INFO:root:Data directory set at: data_goes
DEBUG:root:Temporary directory created at: C:\Users\augus\AppData\Local\Temp\tmp2dwofv_v
INFO:root:Created Aria2 Downloader
INFO:root:TorNet initialized at data_tornet
INFO:root:Data directory set at: data_tornet
DEBUG:root:Temporary directory created at: C:\Users\augus\AppData\Local\Temp\tmp2dwofv_v
INFO:root:Created Aria2 Downloader
INFO:root:Combined initialized at data_combined
INFO:root:Data directory set at: data_combined
DEBUG:root:Temporary directory created at: C:\Users\augus\AppData\Local\Temp\tmp2dwofv_v
INFO:root:Created Aria2 Downloader
INFO:root:Fetching Raw catalog (raw=False) for year(s): None
INFO:root:Loading GOES catalog from CSV...
DEBUG:root:Loaded 129094 records from CSV
INFO:root:Returning Combined catalog with 129094 entries


,type,start_time,end_time,lat,lon,episode_id,event_id,site_name,ef_number,category,az_min,az_max,rng_min,rng_max,filename,GOES_FILENAME,GOES_SATELLITE
0,train,2017-03-01 15:51:00+00:00,2017-03-01 16:06:00+00:00,37.377700,-81.008028,113548,679759,KFCX,-1.0,NUL,271.0,331.0,45932.0,105932.0,train/2017/NUL_170301_160534_KFCX_679759s_Q0.nc,ABI-L2-MCMIPC/2017/060/15/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
1,train,2017-03-01 15:59:00+00:00,2017-03-01 16:14:00+00:00,37.773164,-80.689291,113548,679759,KFCX,-1.0,NUL,306.0,366.0,60748.0,120748.0,train/2017/NUL_170301_161337_KFCX_679759s_V0.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
2,train,2017-03-01 16:00:00+00:00,2017-03-01 16:15:00+00:00,40.539268,-80.481657,114319,684965,KPBZ,-1.0,NUL,242.0,302.0,-7776.0,52224.0,train/2017/NUL_170301_161523_KPBZ_684965s_P7.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
3,train,2017-03-01 16:11:00+00:00,2017-03-01 16:26:00+00:00,37.022655,-81.461622,113548,679707,KFCX,-1.0,NUL,240.0,300.0,75564.0,135564.0,train/2017/NUL_170301_162553_KFCX_679707s_H1.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
4,train,2017-03-01 16:11:00+00:00,2017-03-01 16:26:00+00:00,37.950221,-80.104371,113548,679759,KFCX,-1.0,NUL,-22.0,38.0,73712.0,133712.0,train/2017/NUL_170301_162553_KFCX_679759s_J0.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16


In [3]:
from tornado_helper import GOES

entry = df[df["GOES_SATELLITE"] == "noaa-goes16"].iloc[50]
filename = entry["GOES_FILENAME"]
bucket = entry["GOES_SATELLITE"]

goes = GOES()
file = goes.download(filename, bucket)

INFO:root:GOES initialized at data_goes
INFO:root:Data directory set at: data_goes
DEBUG:root:Temporary directory created at: C:\Users\augus\AppData\Local\Temp\tmp2dwofv_v
INFO:root:Created Aria2 Downloader
DEBUG:root:Downloading file ABI-L2-MCMIPC/2017/060/17/OR_ABI-L2-MCMIPC-M3_G16_s20170601757509_e20170601800282_c20170601800379.nc from bucket noaa-goes16
DEBUG:root:Dependency 'aria2c' found.
DEBUG:root:Checking if Aria2 is running as server
DEBUG:root:Polling socket
DEBUG:root:Aria is not running, starting
INFO:root:Downloading 1 files with Aria2
INFO:root:Downloading 64412650 bytes


100%|██████████| 64.4M/64.4M [00:39<00:00, 1.61MB/s]

INFO:root:Successfully downloaded 1 files
INFO:root:Unzipping files...
DEBUG:root:Unzipping list of files...
DEBUG:root:File does not need to be unzipped
DEBUG:root:Unzipped list of files


In [4]:
from pyproj import Proj
import numpy as np
import netCDF4
import xarray as xr 
import folium

broad_us_coords = [39.8283, -98.5795]

def make_map(lat_1, lat_2, lon_1, lon_2): 
    # Ensure lat_1 < lat_2 and lon_1 < lon_2
    bounds = [[min(lat_1, lat_2), min(lon_1, lon_2)],
              [max(lat_1, lat_2), max(lon_1, lon_2)]]
    
    m = folium.Map(location=broad_us_coords, zoom_start=4)
    folium.Rectangle(bounds=bounds, color="blue", fill=True, fill_opacity=0.2).add_to(m)
    
    return m

def get_ds_coords(ds):
    x = ds['x'].values
    y = ds['y'].values

    proj_attrs = ds['goes_imager_projection'].attrs
    h = proj_attrs['perspective_point_height']
    lon_0 = proj_attrs['longitude_of_projection_origin']
    sweep = proj_attrs.get('sweep_angle_axis', 'x')
    a = proj_attrs.get('semi_major_axis', 6378137.0)

    X, Y = np.meshgrid(x, y)

    p = Proj(proj='geos', h=h, lon_0=lon_0, sweep=sweep, a=a)
    lon, lat = p(X * h, Y * h, inverse=True)

    return [np.nanmin(lat), np.nanmax(lat), np.nanmin(lon), np.nanmax(lon)]

# Raw NOAA Coords
ds = xr.open_dataset(file[0])
[lat_min, lat_max, lon_min, lon_max] = get_ds_coords(ds)

make_map(lat_min, lat_max, lon_min, lon_max)

In [6]:
# TorNet Coords
lat = entry["lat"]
lon = entry["lon"]

make_map(lat + 1, lat - 1, lon + 1, lon - 1)

In [7]:
from pyproj import Proj
import numpy as np

def latlon_to_xy(ds, lat, lon):
    attrs = ds['goes_imager_projection'].attrs  # ✅ this works with xarray

    h = attrs['perspective_point_height']
    lon_0 = attrs['longitude_of_projection_origin']
    sweep = attrs.get('sweep_angle_axis', 'x')
    a = attrs.get('semi_major_axis', 6378137.0)  # fallback to WGS84 if missing

    p = Proj(proj='geos', h=h, lon_0=lon_0, sweep=sweep, a=a)
    x, y = p(lon, lat, inverse=False)
    return x / h, y / h

def clip_ds_to_coord(ds, lat, lon, buffer=1.0):
    print(f"Target: lat={lat}, lon={lon}")
    [lat_min, lat_max, lon_min, lon_max] = get_ds_coords(ds)
    print(f"Dataset bounds:\nLat: {lat_min:.2f} to {lat_max:.2f}\nLon: {lon_min:.2f} to {lon_max:.2f}")

    try:
        x1, y1 = latlon_to_xy(ds, lat + buffer, lon - buffer)
        x2, y2 = latlon_to_xy(ds, lat - buffer, lon + buffer)
    except Exception as e:
        print(f"Projection failed: {e}")
        return None

    x_slice = slice(min(x1, x2), max(x1, x2))
    y_slice = slice(max(y1, y2), min(y1, y2))  # flipped on purpose

    print(f"Clipped dims: x={ds.sel(x=x_slice).sizes.get('x', 0)}, y={ds.sel(y=y_slice).sizes.get('y', 0)}")

    try:
        clipped = ds.sel(x=x_slice, y=y_slice)
        return clipped
    except KeyError:
        print("x/y not found in dataset dimensions.")
        return None
    
# Clipped Cooords
clipped = clip_ds_to_coord(ds, lat, lon)

[lat_min, lat_max, lon_min, lon_max] = get_ds_coords(clipped)
make_map(lat_min, lat_max, lon_min, lon_max)


Target: lat=35.79829185302925, lon=-84.92386450890632
Dataset bounds:
Lat: 13.91 to 52.32
Lon: -139.96 to -49.55
Clipped dims: x=92, y=81


In [8]:
print(clipped)
clipped.to_netcdf("data_combined/cleaned.nc")

<xarray.Dataset> Size: 955kB
Dimensions:                                 (y: 81, x: 92,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates: (12/37)
    t                                       datetime64[ns] 8B ...
  * y                                       (y) float32 324B 0.1027 ... 0.0982
  * x                                       (x) float32 368B 0.008596 ... 0.0...
    y_image                                 float32 4B ...
    x_image                                 float32 4B ...
    band_wavelength_C01                     (band) float32 4B ...
    ...                                      ...
    band_id_C11                             (band) int8 1B ...
    band_id_C12                             (band) int8 1B ...
    band_id_C13                             (band) int8 1B ...
    band_id_C14                             (band) int8 1B ...
    band_id_C15             

C:\Users\augus\AppData\Local\Temp\ipykernel_27412\2413514516.py:2: SerializationWarning: saving variable y with floating point data as an integer dtype without any _FillValue to use for NaNs
  clipped.to_netcdf("data_combined/cleaned.nc")
C:\Users\augus\AppData\Local\Temp\ipykernel_27412\2413514516.py:2: SerializationWarning: saving variable x with floating point data as an integer dtype without any _FillValue to use for NaNs
  clipped.to_netcdf("data_combined/cleaned.nc")
